In [ ]:
%%capture

# Intel® Extension for Scikit-learn installation:
!pip install scikit-learn-intelex -q
!pip install deslib -q

import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import time
import warnings

from deslib.des.knora_e import KNORAE
from deslib.des.knora_u import KNORAU
from scipy.stats import mode
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder

from sklearnex import patch_sklearn
patch_sklearn()

# Mute warnings
warnings.filterwarnings("ignore")

Observation:
- Based on experiments with [T-SNE/LDA/UMAP/](https://www.kaggle.com/remekkinas/super-learner-ensemble-extree-tuned-lda-umap) presented in my previous [notebook - SUPER LEARNER ENSEMBLE - eXTree (TUNED) - LDA/UMAP](https://www.kaggle.com/remekkinas/super-learner-ensemble-extree-tuned-lda-umap) I notice that observations in dataset present some patterns. Hypothesis: neighborhood based algorithm could provide solution to find better model.

**Dynamic Selection (DS)** refers to techniques in which the base classifiers are selected on the fly, according to each new sample to be classified. Only the most competent, or an ensemble containing the most competent classifiers is selected to predict the label of a specific test sample. The rationale for such techniques is that not every classifier in the pool is an expert in classifying all unknown samples; rather, each base classifier is an expert in a different local region of the feature space. (Source: [DESlib documentation](https://deslib.readthedocs.io/en/latest/)

The **Dynamic Ensemble Library**, or **DESlib** for short, is a Python machine learning library that provides an implementation of many different dynamic classifier and dynamic ensemble selection algorithms, The DESlib provides an implementation of the KNORA algorithm with each dynamic ensemble selection technique via the **KNORAE** and **KNORAU** classes respectively.

* **KNORA-Eliminate (KNORA-E)**: Ensemble of classifiers that achieves perfect accuracy on the neighborhood of the new example, with a reducing neighborhood size until at least one perfect classifier is located.
* **KNORA-Union (KNORA-U)**: Ensemble of all classifiers that makes at least one correct prediction on the neighborhood with weighted voting and votes proportional to accuracy on the neighborhood.


In [ ]:
cfg = {
    'TARGET' : 'target',
    'N_FOLDS' : 5,
    'RANDOM': 42,
    'SCORING': 'accuracy'
}

In [ ]:
test = pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv", index_col="row_id")
train = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv", index_col="row_id")
sub = pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")

FEATURES = [col for col in train.columns if col not in ['row_id', cfg['TARGET']]]

In [ ]:
train.drop_duplicates(keep='first', inplace=True)
train = train.reset_index(drop = True)

In [ ]:
lb = LabelEncoder()
y = lb.fit_transform(train['target'])
train['target'] = y

In [ ]:
cv = StratifiedKFold(n_splits = cfg['N_FOLDS'],  random_state = cfg['RANDOM'])

In [ ]:
model_knorae = KNORAE()

n_scores = cross_val_score(model_knorae, train[FEATURES].values, y, scoring= cfg['SCORING'], cv = cv, n_jobs = -1)
print('Mean Accuracy: %.5f (%.5f)' % (np.mean(n_scores), np.std(n_scores)))

model_knorae.fit(train[FEATURES].values, y)

In [ ]:
model_knorau = KNORAU()

n_scores = cross_val_score(model_knorau, train[FEATURES].values, y, scoring = cfg['SCORING'], cv = cv, n_jobs = -1)
print('Mean Accuracy: %.5f (%.5f)' % (np.mean(n_scores), np.std(n_scores)))

model_knorau.fit(train[FEATURES].values, y)

In [ ]:
sub.target = lb.inverse_transform(model_knorae.predict(test[FEATURES].values))
sub.to_csv("KNORAE-submission.csv", index=False)
sub.head(10)

In [ ]:
sub.target = lb.inverse_transform(model_knorau.predict(test[FEATURES].values))
sub.to_csv("KNORAU-submission.csv", index=False)
sub.head(10)